In [2]:
# Author: Jeffrey Robert Lynch
# Run this cell to launch the GUI. The GUI features buttons for executing all application functions. 
# Allow 5 seconds or so for the GUI to launch. The model and test data, 1000 images, must be loaded before the GUI launches.
# Running cells beyond this one is not advised, as they are used for model training and manipulating data not used by the GUI.
# These cells were included for documentation. 
# Import Required libraries for model training, data exploration, model anaylsis, and prediction functions.
import os
# Disable GPU. Previous sustained high GPU usage caused overheating.
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Class names.
diseaseClasses = ['Bacterial_Spot', 'Early_Blight', 'Late_Blight', 'Leaf_Mold', 'Septoria_Spot', 'Spider_Mites',
                      'Target_Spot', 'Yellow_Curl', 'Mosaic_Virus', 'Healthy']

# Load final model, model 3.
modelVersion = 3
modelPath = f"../models/{modelVersion}"
loadedModel = load_model(modelPath)
model = loadedModel
print(f"Model 3 Loaded.")

# Load final model, model 3.
#modelVersion = 3
#modelPath = f"../models/my_model.h5"
#loadedModel = load_model(modelPath)
#model = loadedModel
#print(f"Model 3 Loaded.")

# Secondary Set.
secondarySet = tf.keras.preprocessing.image_dataset_from_directory(
    'val',
    shuffle=True,
    image_size = (256, 256),
    batch_size = 32
)

# Display Class Images Function. 
# Displays 24 representative images from the Secondary Set, along with their true labels.
def displayClassImages(button):
    output.clear_output(wait=True)
    with output:
        for imageBatch, labelBatch in secondarySet.take(1):
            plt.figure(figsize=(15,15))
            for i in range(24):
                grid = plt.subplot(4, 6, i + 1)
                plt.imshow(imageBatch[i].numpy().astype("uint8"))
                plt.title(diseaseClasses[labelBatch[i]])
                plt.axis("off")
            plt.show()

# Generate Class Distribution Chart Function.
# Generates a Class Distribution Chart for the Secondary Set. 
def generateClassDistribution(button):
    output.clear_output(wait=True)
    with output:
        classCounts = {}
        for diseaseClass in os.listdir('val'):
            classDir = os.path.join('val', diseaseClass)
            if os.path.isdir(classDir):
                numberImages = len(os.listdir(classDir))
                classCounts[diseaseClass] = numberImages
        customColors = ['brown', 'cyan', 'green', 'blue', 'orange', 'red', 'gray', 'purple', 'magenta', 'yellow']
        plt.figure(figsize=(10, 6))
        plt.bar(classCounts.keys(), classCounts.values(), color=customColors)
        plt.xlabel('Class')
        plt.ylabel('Number of Images')
        plt.title('Class Distribution')
        plt.xticks(rotation=45, ha='right')  
        plt.show()

# Evaluate Model Accuracy Function.
# Evaluates the accuracy of the model using the Preliminary Set and the Secondary Set.
# Both image sets are of equivalent size and class distribution. Mean accuracy is determined and returned.
def evaluateModelAccuracy(button):
    output.clear_output(wait=True)
    with output:
        # Preliminary accuracy. Accuracy value input manually to prevent the need for an additional large dataset in evaluation.
        # Preliminary accuracy confirmed in screenshot Figure D5.
        #scorePreliminary = model.evaluate(preliminarySet)
        #accuracyPreliminary = scorePreliminary[1]
        accuracyPreliminary = 0.955078125 
        print(f"Preliminary Model Accuracy: {accuracyPreliminary}")
        # Secondary accuracy.
        scoreSecondary = model.evaluate(secondarySet)
        accuracySecondary = scoreSecondary[1]  
        print(f"Secondary Model Accuracy: {accuracySecondary}")
        # Mean accuracy.
        meanAccuracy = (accuracyPreliminary + accuracySecondary) / 2
        print(f"Mean Model Accuracy: {meanAccuracy}")

# Generate Confusion Matrix Function.
# Generates a Confusion Matrix for the model using the Secondary Set.
def generateConfusionMatrix(button):
    output.clear_output(wait=True)
    with output:
        yTrue = []
        yPredict = []
        for images, labels in secondarySet:
            predictions = model.predict(images)
            predictLabels = np.argmax(predictions, axis=1)
            yTrue.extend(labels.numpy())
            yPredict.extend(predictLabels)

        confusionMatrix = confusion_matrix(yTrue, yPredict)
        plt.figure(figsize=(12, 10)) 
        sns.heatmap(confusionMatrix, annot=True, cmap='Greens', fmt='d', 
                    xticklabels=diseaseClasses, yticklabels=diseaseClasses, 
                    cbar_kws={'label': 'Frequency'})
        plt.xticks(rotation=45, ha='right')
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()

# Prediction Function.
def predict(model, img):
    imgArray = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    imgArray = tf.expand_dims(imgArray, 0)
    preds = model.predict(imgArray)
    predictClass = diseaseClasses[np.argmax(preds[0])]
    return predictClass

# Alternate Prediction Function. 
def predictA(model, img):
    imgArray = tf.keras.preprocessing.image.img_to_array(img)
    imgArray = tf.expand_dims(imgArray, 0)
    preds = model.predict(imgArray)
    predictClass = diseaseClasses[np.argmax(preds[0])]
    predictConf = round(100 * (np.max(preds[0])), 2)
    return predictClass, predictConf

# Display Batch Predictions Function.
# Displays 24 images from the Secondary Set accompanied by true label, predicted label, and confidence.
def displayBatchPredictions(button):
    output.clear_output(wait=True)
    with output:
        plt.figure(figsize=(15, 15))
        correctPred = 0
        totalPred = 0
        for images, labels in secondarySet.take(1):
            for i in range(24):
                grid = plt.subplot(4, 6, i + 1)
                plt.imshow(images[i].numpy().astype("uint8"))
                predictClass, predictConf = predictA(model, images[i].numpy())
                actualClass = diseaseClasses[labels[i]]
                plt.title(f"True: {actualClass},\n Predict: {predictClass}\n Confidence: {predictConf}%")
                plt.axis("off")
                if actualClass == predictClass:
                    correctPred += 1
                totalPred += 1
        accuracy = correctPred / totalPred if totalPred > 0 else 0
        print(f"Accuracy: {accuracy:.2%}")
        plt.show()
        
# Map Field Disease Function.
# Display a color-coded field map representing disease locations within a field. 
# This function was designed for use with field test data from a 24 plant field test garden. 
# The plants are currently too immature for testing and will not be viable till July or August.
# The Secondary Set has been substituted for simulation.
def mapFieldDisease(button):
    output.clear_output(wait=True)
    numberRows = 4
    numberColumns = 6
    field = np.zeros((numberRows, numberColumns))
    with output:
        idx = 0
        for i in range(numberRows):
            for j in range(numberColumns):
                if idx < len(secondarySet):
                    image, _ = next(iter(secondarySet.skip(idx).take(1)))  
                    predictProbs = model.predict(image)[0]
                    predictLabel = np.argmax(predictProbs)  
                    field[i, j] = predictLabel
                    idx += 1

        classColors = ['brown', 'cyan', 'blue', 'orange', 'red', 'gray', 'purple', 'yellow', 'magenta', 'green']
        plt.figure(figsize=(15, 10))
        plt.imshow(field, cmap=plt.matplotlib.colors.ListedColormap(classColors), interpolation='nearest', aspect='equal')
        cbar = plt.colorbar(label='Class Label', ticks=np.arange(len(diseaseClasses)))
        cbar.set_ticklabels(diseaseClasses) 
        plt.title('Field Map')
        plt.axis("off")
        plt.xlabel('Columns')
        plt.ylabel('Rows')
        plt.show()

# Load Model 3.
def loadModel3(button):
    modelVersion = 3
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 3 Loaded.")

# Load Model 2.
def loadModel2(button):
    modelVersion = 2
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 2 Loaded.")

# Load Model 1.
def loadModel1(button):
    modelVersion = 1
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 1 Loaded.")

# Display Class Images Function. 
classImagesButton = widgets.Button(description="Class Images")
classImagesButton.on_click(displayClassImages)

# Generate Class Distribution Chart Function.
classDistributionButton = widgets.Button(description="Class Distribution")
classDistributionButton.on_click(generateClassDistribution)

# Evaluate Model Accuracy Function.
evaluateAccuracyButton = widgets.Button(description="Evaluate Accuracy")
evaluateAccuracyButton.on_click(evaluateModelAccuracy)

# Generate Confusion Matrix Function.
confusionMatrixButton = widgets.Button(description="Confusion Matrix")
confusionMatrixButton.on_click(generateConfusionMatrix)

# Display Batch Predictions Function.
batchPredictionsButton = widgets.Button(description="Batch Predictions")
batchPredictionsButton.on_click(displayBatchPredictions)

# Map Field Disease Function.
fieldMapButton = widgets.Button(description="Map Field Disease")
fieldMapButton.on_click(mapFieldDisease)

# Load Model 3.
loadModel3Button = widgets.Button(description="Load Model 3")
loadModel3Button.on_click(loadModel3) 

# Load Model 2.
loadModel2Button = widgets.Button(description="Load Model 2")
loadModel2Button.on_click(loadModel2) 

# Load Model 1.
loadModel1Button = widgets.Button(description="Load Model 1")
loadModel1Button.on_click(loadModel1) 

# Disable Load Model Buttons for evaluation.
loadModel3Button.disable = True
loadModel2Button.disable = True
loadModel1Button.disable = True

# Text Elements.
headerInfo = widgets.HTML("<h2>C964: CNN-based Automated Crop Health Evaluator</h2>")
summaryInfo1 = widgets.HTML("<p>NAT’s tomato-producing clients lose an estimated 23% of their potential tomato yields to nine common diseases. Timely intervention is required to mitigate the damage caused by these diseases. Early, accurate diagnosis is critical to applying the right intervention because corrective action is highly differential.</p>") 
summaryInfo2 = widgets.HTML("<p>Nimble Agricultural Technologies needed a CNN-based, deep learning solution to accurately detect and diagnose these diseases in tomato plant leaf images. A successful model was defined as a model achieving a mean accuracy of 90% or higher classifying 10 disease states. Mean accuracy was defined as the model’s average accuracy after testing two different, but equivalent, test sets. </p>")
dataInfo = widgets.HTML("<p>The Secondary Test Set contains 1000 images of tomato plant leaves divided into 10 disease class states. These are the 9 diseases most commonly affecting NAT's customers and a healthy state. See Figure B1 for more information about specific diseases.</p>")
modelInfo = widgets.HTML("<p>Model 3, the project's final model, can classify this set with approximately 94.5% accuracy. Model accuracy can be confirmed using the functions below. The buttons explore the data, analyze the model's classification capabilities, and use the model to classify images from the set.</p>") 
instructionInfo = widgets.HTML("<p>Thank you for evaluating the model! Display function batch sizes have been limited to 24 images for easier evaluation. Load Model functions are disabled due to submission size constraints. The Secondary Test Set and Model 3 have already been loaded. Image and map functions provide dynamic results, returning a new batch of images each time the button is pressed.</p>")
classImagesDescription = widgets.HTML("<p>Display 24 representative images from the Secondary Test Set with true class labels.</p>")
classDistributionDescription = widgets.HTML("<p>Generate Class Distribution Chart for the Secondary Test Set.</p>")
evaluateAccuracyDescription = widgets.HTML("<p>Evaluate Model 3's accuracy classifying the Secondary Test Set and return additional accuracy metrics.</p>")
confusionMatrixDescription = widgets.HTML("<p>Generate a Confusion Matrix to analyze the model's Secondary Test Set classifications in detail.</p>")
batchPredictionsDescription = widgets.HTML("<p>Display 24 images from the Secondary Test Set with true class, model prediction, and model confidence. Prediction accuracy for batch is displayed.</p>")
fieldMapDescription = widgets.HTML("<p>Display a color-coded field map representing disease locations within a field. Secondary Test Set used to simulate a 24 plant field test garden.</p>")
loadModel3Description = widgets.HTML("<p>Load Model 3. Disabled for evaluation. Model 3 is loaded by default.</p>")
loadModel2Description = widgets.HTML("<p>Load Model 2. Disabled for evaluation. Model 3 is loaded by default.</p>")
loadModel1Description = widgets.HTML("<p>Load Model 1. Disabled for evaluation. Model 3 is loaded by default.</p>")

# Output widget and GUI.
output = widgets.Output()
display(headerInfo)
display(summaryInfo1)
display(summaryInfo2)
display(dataInfo)
display(modelInfo)
display(instructionInfo)
display(widgets.VBox([
    widgets.HBox([classImagesButton, classImagesDescription]),
    widgets.HBox([classDistributionButton, classDistributionDescription]),
    widgets.HBox([evaluateAccuracyButton, evaluateAccuracyDescription]),
    widgets.HBox([confusionMatrixButton, confusionMatrixDescription]),
    widgets.HBox([batchPredictionsButton, batchPredictionsDescription]),
    widgets.HBox([fieldMapButton, fieldMapDescription]),
    widgets.HBox([loadModel3Button, loadModel3Description]),
    widgets.HBox([loadModel2Button, loadModel2Description]),
    widgets.HBox([loadModel1Button, loadModel1Description]),
    output
]))

Model 3 Loaded.
Found 1000 files belonging to 10 classes.


HTML(value='<h2>C964: CNN-based Automated Crop Health Evaluator</h2>')

HTML(value='<p>NAT’s tomato-producing clients lose an estimated 23% of their potential tomato yields to nine c…

HTML(value='<p>Nimble Agricultural Technologies needed a CNN-based, deep learning solution to accurately detec…

HTML(value="<p>The Secondary Test Set contains 1000 images of tomato plant leaves divided into 10 disease clas…

HTML(value="<p>Model 3, the project's final model, can classify this set with approximately 94.5% accuracy. Mo…

HTML(value='<p>Thank you for evaluating the model! Display function batch sizes have been limited to 24 images…

In [ ]:
# The following cells contain code used for model training or data exploration not related to the application. 
# Running this cell, or other following cells, will cause issues with the application.
# Import Required libraries for model training, data exploration, model anaylsis, and prediction functions.
#import os
# Disable GPU. Previous sustained high GPU usage caused overheating.
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Development Dataset. Split for training, validation, and preliminary accuracy evaluation.
developmentSet = tf.keras.preprocessing.image_dataset_from_directory(
    'training',
    shuffle=True,
    image_size = (256, 256),
    batch_size = 32
)

# 10,000 images
# 8,000 training
# 1,000 validation
# 1,000 testing prelim
trainSetSize = 0.8
trainSet = developmentSet.take(250)
preliminarySet = developmentSet.skip(250)
validationSetSize = 0.1
validationSet = preliminarySet.take(31)
preliminarySet = preliminarySet.skip(31)

def get_dataset_partitions_tf(developmentSet, trainSetSplit = 0.8, validationSetSplit = 0.1, testSetSplit = 0.1, shuffle = True, shuffleSize = 50000):
    developmentSetSize = len(developmentSet)
    if shuffle:
        developmentSet = developmentSet.shuffle(shuffleSize, seed = 67)
    trainSetSize = int(trainSetSplit * developmentSetSize)
    validationSetSize = int(validationSetSplit * developmentSetSize)
    trainSet = developmentSet.take(trainSetSize)
    validationSet = developmentSet.skip(trainSetSize).take(validationSetSize)
    preliminarySet = developmentSet.skip(trainSetSize).skip(validationSetSize)
    return trainSet, validationSet, preliminarySet

trainSet, validationSet, preliminarySet = get_dataset_partitions_tf(developmentSet)

In [ ]:
# Do not run this cell if using the application. These functions are intended for use with a different data set.
# Running this cell while using the application will cause issues with the application.
# Import Required libraries for model training, data exploration, model anaylsis, and prediction functions.
import os
# Disable GPU. Previous sustained high GPU usage caused overheating.
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Class names.
diseaseClasses = ['Bacterial_Spot', 'Early_Blight', 'Late_Blight', 'Leaf_Mold', 'Septoria_Spot', 'Spider_Mites',
                       'Target_Spot', 'Yellow_Curl', 'Mosaic_Virus', 'Healthy']

# Load final model, model 3.
modelVersion = 3
modelPath = f"../models/{modelVersion}"
loadedModel = load_model(modelPath)
model = loadedModel
print(f"Model 3 Loaded.")

# Secondary Set.
secondarySet = tf.keras.preprocessing.image_dataset_from_directory(
    'val',
    shuffle=True,
    image_size = (256, 256),
    batch_size = 32
)

# Display Class Images Function. 
# Displays 24 representative images from the Set, along with their true labels.
def displayClassImagesP(button):
    output.clear_output(wait=True)
    with output:
        for imageBatch, labelBatch in preliminarySet.take(1):
            plt.figure(figsize=(15,15))
            for i in range(24):
                grid = plt.subplot(4, 6, i + 1)
                plt.imshow(imageBatch[i].numpy().astype("uint8"))
                plt.title(diseaseClasses[labelBatch[i]])
                plt.axis("off")
            plt.show()

# Generate Class Distribution Chart Function.
# Generates a Class Distribution Chart for the Set. 
def generateClassDistributionP(button):
    output.clear_output(wait=True)
    with output:
        classCounts = {}
        for diseaseClass in os.listdir('training'):
            classDir = os.path.join('training', diseaseClass)
            if os.path.isdir(classDir):
                numberImages = len(os.listdir(classDir))
                classCounts[diseaseClass] = numberImages
        customColors = ['brown', 'cyan', 'green', 'blue', 'orange', 'red', 'gray', 'purple', 'magenta', 'yellow']
        plt.figure(figsize=(10, 6))
        plt.bar(classCounts.keys(), classCounts.values(), color=customColors)
        plt.xlabel('Class')
        plt.ylabel('Number of Images')
        plt.title('Class Distribution')
        plt.xticks(rotation=45, ha='right')  
        plt.show()

# Evaluate Model Accuracy Function.
# Evaluates the accuracy of the model using the Preliminary Set and the Secondary Set.
# Both image sets are of equivalent size and class distribution. Mean accuracy is determined and returned.
def evaluateModelAccuracyP(button):
    output.clear_output(wait=True)
    with output:
        # Preliminary accuracy. Accuracy value input manually to prevent the need for an additional large dataset in evaluation.
        # Preliminary accuracy confirmed in screenshot Figure D5.
        #scorePreliminary = model.evaluate(preliminarySet)
        #accuracyPreliminary = scorePreliminary[1]
        accuracyPreliminary = 0.955078125 
        print(f"Preliminary Model Accuracy: {accuracyPreliminary}")
        # Secondary accuracy.
        scoreSecondary = model.evaluate(secondarySet)
        accuracySecondary = scoreSecondary[1]  
        print(f"Secondary Model Accuracy: {accuracySecondary}")
        # Mean accuracy.
        meanAccuracy = (accuracyPreliminary + accuracySecondary) / 2
        print(f"Mean Model Accuracy: {meanAccuracy}")

# Generate Confusion Matrix Function.
# Generates a Confusion Matrix for the model using the Set.
def generateConfusionMatrixP(button):
    output.clear_output(wait=True)
    with output:
        yTrue = []
        yPredict = []
        for images, labels in preliminarySet:
            predictions = model.predict(images)
            predictLabels = np.argmax(predictions, axis=1)
            yTrue.extend(labels.numpy())
            yPredict.extend(predictLabels)

        confusionMatrix = confusion_matrix(yTrue, yPredict)
        plt.figure(figsize=(12, 10)) 
        sns.heatmap(confusionMatrix, annot=True, cmap='Greens', fmt='d', 
                    xticklabels=diseaseClasses, yticklabels=diseaseClasses, 
                    cbar_kws={'label': 'Frequency'})
        plt.xticks(rotation=45, ha='right')
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()

# Prediction Function.
def predict(model, img):
    imgArray = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    imgArray = tf.expand_dims(imgArray, 0)
    preds = model.predict(imgArray)
    predictClass = diseaseClasses[np.argmax(preds[0])]
    return predictClass

# Alternate Prediction Function. 
def predictA(model, img):
    imgArray = tf.keras.preprocessing.image.img_to_array(img)
    imgArray = tf.expand_dims(imgArray, 0)
    preds = model.predict(imgArray)
    predictClass = diseaseClasses[np.argmax(preds[0])]
    predictConf = round(100 * (np.max(preds[0])), 2)
    return predictClass, predictConf

# Display Batch Predictions Function.
# Displays 24 images from the Set accompanied by true label, predicted label, and confidence.
def displayBatchPredictionsP(button):
    output.clear_output(wait=True)
    with output:
        plt.figure(figsize=(15, 15))
        for images, labels in preliminarySet.take(1):
            for i in range(24):
                grid = plt.subplot(4, 6, i + 1)
                plt.imshow(images[i].numpy().astype("uint8"))
                predictClass, predictConf = predictA(model, images[i].numpy())
                actualClass = diseaseClasses[labels[i]]
                plt.title(f"True: {actualClass},\n Predict: {predictClass}\n Confidence: {predictConf}%")
                plt.axis("off")
        plt.show()

        
# Map Field Disease Function.
# Display a color-coded field map representing disease locations within a field. 
# This function was designed for use with field test data from a 24 plant field test garden. 
# The plants are currently too immature for testing and will not be viable till July or August.
# The Set has been substituted for simulation.
def mapFieldDiseaseP(button):
    output.clear_output(wait=True)
    numberRows = 4
    numberColumns = 6
    field = np.zeros((numberRows, numberColumns))
    with output:
        idx = 0
        for i in range(numberRows):
            for j in range(numberColumns):
                if idx < len(preliminarySet):
                    image, _ = next(iter(preliminarySet.skip(idx).take(1)))  
                    predictProbs = model.predict(image)[0]
                    predictLabel = np.argmax(predictProbs)  
                    field[i, j] = predictLabel
                    idx += 1

        classColors = ['brown', 'cyan', 'blue', 'orange', 'red', 'gray', 'purple', 'yellow', 'magenta', 'green']
        plt.figure(figsize=(15, 10))
        plt.imshow(field, cmap=plt.matplotlib.colors.ListedColormap(classColors), interpolation='nearest', aspect='auto')
        cbar = plt.colorbar(label='Class Label', ticks=np.arange(len(diseaseClasses)))
        cbar.set_ticklabels(diseaseClasses) 
        plt.title('Field Map')
        plt.axis("off")
        plt.xlabel('Columns')
        plt.ylabel('Rows')
        plt.show()

# Load Model 3.
def loadModel3P(button):
    modelVersion = 3
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 3 Loaded.")

# Load Model 2.
def loadModel2P(button):
    modelVersion = 2
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 2 Loaded.")

# Load Model 1.
def loadModel1P(button):
    modelVersion = 1
    modelPath = f"../models/{modelVersion}"
    loadedModel = load_model(modelPath)
    model = loadedModel
    print(f"Model 1 Loaded.")

# Display Class Images Function. 
classImagesButton = widgets.Button(description="Class Images")
classImagesButton.on_click(displayClassImagesP)

# Generate Class Distribution Chart Function.
classDistributionButton = widgets.Button(description="Class Distribution")
classDistributionButton.on_click(generateClassDistributionP)

# Evaluate Model Accuracy Function.
evaluateAccuracyButton = widgets.Button(description="Evaluate Accuracy")
evaluateAccuracyButton.on_click(evaluateModelAccuracyP)

# Generate Confusion Matrix Function.
confusionMatrixButton = widgets.Button(description="Confusion Matrix")
confusionMatrixButton.on_click(generateConfusionMatrixP)

# Display Batch Predictions Function.
batchPredictionsButton = widgets.Button(description="Batch Predictions")
batchPredictionsButton.on_click(displayBatchPredictionsP)

# Map Field Disease Function.
fieldMapButton = widgets.Button(description="Map Field Disease")
fieldMapButton.on_click(mapFieldDiseaseP)

# Load Model 3.
loadModel3Button = widgets.Button(description="Load Model 3")
loadModel3Button.on_click(loadModel3P) 

# Load Model 2.
loadModel2Button = widgets.Button(description="Load Model 2")
loadModel2Button.on_click(loadModel2P) 

# Load Model 1.
loadModel1Button = widgets.Button(description="Load Model 1")
loadModel1Button.on_click(loadModel1P) 

# Disable Load Model Buttons for evaluation.
loadModel3Button.disable = True
loadModel2Button.disable = True
loadModel1Button.disable = True

# Text Elements.
headerInfo = widgets.HTML("<h2>C964: CNN-based Automated Crop Health Evaluator</h2>")
summaryInfo1 = widgets.HTML("<p>NAT’s tomato-producing clients lose an estimated 23% of their potential tomato yields to nine common diseases. Timely intervention is required to mitigate the damage caused by these diseases. Early, accurate diagnosis is critical to applying the right intervention because corrective action is highly differential.</p>") 
summaryInfo2 = widgets.HTML("<p>Nimble Agricultural Technologies needed a CNN-based, deep learning solution to accurately detect and diagnose these diseases in tomato plant leaf images. A successful model was defined as a model achieving a mean accuracy of 90% or higher classifying 10 disease states. Mean accuracy was defined as the model’s average accuracy after testing two different, but equivalent, test sets. </p>")
dataInfo = widgets.HTML("<p>The Secondary Test Set contains 1000 images of tomato plant leaves divided into 10 disease class states. These are the 9 diseases most commonly affecting NAT's customers and a healthy state. See Figure B1 for more information about specific diseases.</p>")
modelInfo = widgets.HTML("<p>Model 3, the project's final model, can classify this set with approximately 94.5% accuracy. The buttons below explore the data, analyze the model's classification capabilities, and use the model to classify images from the set.</p>") 
instructionInfo = widgets.HTML("<p>Thank you for evaluating the model! Display function batch sizes have been limited to 24 images for easier evaluation. Load Model functions are disabled due to submission size constraints. The Secondary Test Set and Model 3 have already been loaded. Please select an option.</p>")
classImagesDescription = widgets.HTML("<p>Display 24 representative images from the Secondary Test Set with true class labels.</p>")
classDistributionDescription = widgets.HTML("<p>Generate Class Distribution Chart for the Secondary Test Set.</p>")
evaluateAccuracyDescription = widgets.HTML("<p>Evaluate Model 3's accuracy classifying the Secondary Test Set and return additional accuracy metrics.</p>")
confusionMatrixDescription = widgets.HTML("<p>Generate a Confusion Matrix to analyze the model's Secondary Test Set classifications in detail.</p>")
batchPredictionsDescription = widgets.HTML("<p>Display 24 images from the Secondary Test Set with true class, model prediction, and model confidence.</p>")
fieldMapDescription = widgets.HTML("<p>Display a color-coded field map representing disease locations within a field. Secondary Test Set used to simulate 24 plant field test garden.</p>")
loadModel3Description = widgets.HTML("<p>Load Model 3. Disabled for evaluation. Model 3 is loaded by default.</p>")
loadModel2Description = widgets.HTML("<p>Load Model 2. Disabled for evaluation. Model 3 is loaded by default.</p>")
loadModel1Description = widgets.HTML("<p>Load Model 1. Disabled for evaluation. Model 3 is loaded by default.</p>")

# Output widget and GUI.
output = widgets.Output()
display(headerInfo)
display(summaryInfo1)
display(summaryInfo2)
display(dataInfo)
display(modelInfo)
display(instructionInfo)
display(widgets.VBox([
    widgets.HBox([classImagesButton, classImagesDescription]),
    widgets.HBox([classDistributionButton, classDistributionDescription]),
    widgets.HBox([evaluateAccuracyButton, evaluateAccuracyDescription]),
    widgets.HBox([confusionMatrixButton, confusionMatrixDescription]),
    widgets.HBox([batchPredictionsButton, batchPredictionsDescription]),
    widgets.HBox([fieldMapButton, fieldMapDescription]),
    widgets.HBox([loadModel3Button, loadModel3Description]),
    widgets.HBox([loadModel2Button, loadModel2Description]),
    widgets.HBox([loadModel1Button, loadModel1Description]),
    output
]))

In [ ]:
# Resize, rescale, normalize, and noise. Training functions disabled to prevent overwriting model.
#preprocessSet = tf.keras.Sequential([
    #layers.experimental.preprocessing.Resizing(256, 256),
    #layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [ ]:
# Apply flipping, rotation, cutout, etc. to augment the training set. Training functions disabled to prevent overwriting model.
#augmentSet = tf.keras.Sequential([
    #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    #layers.experimental.preprocessing.RandomRotation(0.2),
    #layers.experimental.preprocessing.RandomCutout(0.2)
    #layers.GaussianBlur(3, sigma=1.0)
#])

In [ ]:
# Architecture for the Convolutional Neural Network. Define shape. Define classes. Define layers & activation. 
# Training functions disabled to prevent overwriting model.
#diseaseClassesSize = 10
#input_shape = (32, 256, 256, 3)
#model = models.Sequential([
    #preprocessSet,
    #augmentSet,
    #layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape),
    #layers.MaxPooling2D((2,2)),
    #layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'),
    #layers.MaxPooling2D((2,2)),
    #layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'),
    #layers.MaxPooling2D((2,2)),
    #layers.Conv2D(64, (3,3), activation = 'relu'),
    #layers.MaxPooling2D((2,2)),
    #layers.Conv2D(64, (3,3), activation = 'relu'),
    #layers.MaxPooling2D((2,2)),
    #layers.Conv2D(64, (3,3), activation = 'relu'),
    #layers.MaxPooling2D((2,2)),
    #layers.Flatten(),
    #layers.Dense(64, activation = 'relu'),
    #layers.Dense(diseaseClassesSize, activation = 'softmax'),
#])

#model.build(input_shape = input_shape)

In [2]:
# Architecture Summary.
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 sequential_10 (Sequential)  (None, 256, 256, 3)       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                

In [ ]:
# Adam for optimization and SparseCategoricalCrossEntropy for loss function. Training functions disabled to prevent overwriting model.
#model.compile(
    #optimizer = 'adam',
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    #metrics = ['accuracy']
#)

In [ ]:
# Begin training with the established parameters. Training functions disabled to prevent overwriting model.
#trainRecord = model.fit(trainSet, epochs = 200, batch_size = 32, verbose = 1, validation_data = validationSet)

In [ ]:
# Preliminary test set. 
#scorePrelim = model.evaluate(preliminarySet)
#scorePrelim

In [ ]:
# Save Trained Model. Training functions disabled to prevent overwriting model.
#model_version = 4
#model.save(f"../models/{model_version}")

In [ ]:
import os
print(os.listdir('val'))

In [ ]:
print(os.listdir('val/Tomato___Spider_mites Two-spotted_spider_mite'))

In [ ]:
img_path = 'val/Tomato___Spider_mites Two-spotted_spider_mite/1bbcbd22-5959-4ae4-a044-85f81a1a02dd___Com.G_SpM_FL 9319.JPG'
img = plt.imread(img_path)
plt.imshow(img)
plt.show()

In [ ]:
import sys
print(sys.version)